In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.2 MB/s eta 0:00:00:00:0100:01


In [8]:
import sys
import os
from typing import List, Tuple, Optional, Dict

from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
from transformers import pipeline
import torch.nn as nn
from sentence_transformers import SentenceTransformer

In [48]:
import torch
import torch.nn.functional as F
import gc # Added for memory management
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

class ModelProvider:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def load(self):
        print(f"📥 Loading tokenizer: {self.model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
        # Ensure padding token exists for batching/safety
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        print("⚡ Enabling 4-bit quantization (NF4)...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )

        print("🧠 Loading model to GPU...")
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto", 
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )

        print(f"✅ Model loaded on: {self.model.device}")
        return self

    def generate_with_logprobs(self, prompt: str, max_new_tokens: int = 150, temperature: float = 0.7):
        # 1. Clear cache before generation to prevent OOM
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()

        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].to(self.model.device)
        input_len = input_ids.shape[1]

        outputs = self.model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            return_dict_in_generate=True,
            output_scores=True,
            pad_token_id=self.tokenizer.pad_token_id
        )

        sequences = outputs.sequences[0]
        scores = outputs.scores
        
        # 2. Decode text safely
        full_text = self.tokenizer.decode(sequences, skip_special_tokens=True)
        # Using a more robust split for the prompt echo
        decoded_prompt = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        generated_text = full_text.replace(decoded_prompt, "").strip()

        # 3. Log Probability Computation with safety zip
        logprobs = []
        gen_token_ids = sequences[input_len:]

        # Use zip to prevent IndexError if EOS token stops generation early
        for step_logits, token_id in zip(scores, gen_token_ids):
            step_logprobs = F.log_softmax(step_logits[0], dim=-1)
            logprobs.append(step_logprobs[token_id].item())

        avg_logprob = sum(logprobs) / len(logprobs) if logprobs else 0.0

        return {
            "text": generated_text,
            "logprobs": logprobs,
            "avg_logprob": avg_logprob
        }

In [34]:


from langchain_core.prompts import ChatPromptTemplate

class PromptLibrary:
    def __init__(self):
        self.supporting_generator_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a prolific debater, your task is to generate arguments on the given topic and position."),
            ("human", "{topic}...")
        ])
        self.opposing_generator_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a prolific debater, your task is to generate arguments against the given topic and position."),
            ("human", "{topic}...")
        ])
        self.nli_critic_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a critic of arguments. Evaluate the relevance of the argument to the premise."),
            ("human", "Premise: {premise}\nArgument: {argument}")
        ])
        self.supporting_merger_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert debater. You have been given supporting arguments  and counter arguments to oppose the opposition on a topic. Your task is to merge them into a single  argument for the topic."),
            ("human", "Supporting Argument: {supporting}\nOpposing Argument: {opposing}")
        ])
        self.opposing_merger_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert debater. You have been given supporting arguments  and counter arguments to oppose the opposition on a topic. Your task is to merge them into a single  argument against the topic."),
            ("human", "Supporting Argument: {supporting}\nOpposing Argument: {opposing}")
        ])
        # ... all other prompts

In [24]:
def nli_relevance_check(nli_pipeline,premise: str, hypothesis: str) -> bool:
    """
    Perform a simple NLI relevance check between the premise and hypothesis.
    This is a placeholder function and should be replaced with an actual NLI model for production use.
    """
    
    
    result = nli_pipeline(f"{premise} [SEP] {hypothesis}")   
    return {
        "label": result[0]['label'],
        "score": result[0]['score']
    }
def similarity_check(model, text1: str, text2: str) -> float:
    # 1. Compute embeddings for both sentences
    # SentenceTransformer.encode returns a numpy array or torch tensor
    embeddings = model.encode([text1, text2], convert_to_tensor=True)
    
    # 2. Compute Cosine Similarity
    # util.cos_sim returns a matrix; we take [0][1] for the score between text1 and text2
    cosine_score = util.cos_sim(embeddings[0], embeddings[1])
    
    # 3. Scale from [-1, 1] to [0, 1]
    scaled_score = (cosine_score.item() + 1) / 2
    
    return scaled_score

In [25]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ibm-research/argument_quality_ranking_30k", "argument_quality_ranking")

In [36]:


def supporting_argument_node(state, model_provider: ModelProvider):
    topic = state["topic"]

    prompt = PromptLibrary().supporting_generator_prompt.format(
        topic=topic
    )

    result = model_provider.generate_with_logprobs(prompt)

    return {
        "argument": result["text"],
        "logprobs": result["logprobs"],
        "avg_logprob": result["avg_logprob"]
    }


def opposing_argument_node(state, model_provider: ModelProvider):
    topic = state["topic"]

    prompt = PromptLibrary().opposing_generator_prompt(
        topic=topic
    )

    result = model_provider.generate_with_logprobs(prompt)

    return {
        "argument": result["text"],
        "logprobs": result["logprobs"],
        "avg_logprob": result["avg_logprob"]
    }


In [50]:
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer


class FeatureExtractor:

    def __init__(self):

        # =====================================================
        # Device Setup
        # =====================================================
        self.device_id = 0 if torch.cuda.is_available() else -1
        self.device_str = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"🚀 FeatureExtractor using device: {self.device_str}")

        # =====================================================
        # NLI Pipeline
        # =====================================================
        self.nli_pipeline = pipeline(
            "text-classification",
            model="roberta-large-mnli",
            device=self.device_id
        )

        # =====================================================
        # Similarity Model
        # =====================================================
        self.similarity_model = SentenceTransformer(
            "all-MiniLM-L6-v2",
            device=self.device_str
        )

        # =====================================================
        # Model Provider (LLM)
        # =====================================================
        self.model_provider = ModelProvider(
            model_name="mistralai/Mistral-7B-Instruct-v0.1",
            
        )

        print("📥 Loading Model Weights & Tokenizer...")
        self.model_provider.load()

    # =========================================================
    # Feature Extraction
    # =========================================================
    def extract_features(self, topic: str):

        # --- Generate statements ---
        supporting_result = supporting_argument_node(
            {"topic": topic},
            self.model_provider
        )

        opposing_result = opposing_argument_node(
            {"topic": topic},
            self.model_provider
        )

        avg_logprob = supporting_result["avg_logprob"]
        argument = supporting_result["argument"]
        opposing_argument = opposing_result["argument"]

        # =====================================================
        # 1️⃣ Relevance (Topic → Support)
        # =====================================================
        relevance_result = nli_relevance_check(
            self.nli_pipeline,
            topic,
            argument
        )

        if relevance_result["label"] == "ENTAILMENT":
            relevance_score_support = relevance_result["score"]
        elif relevance_result["label"] == "CONTRADICTION":
            relevance_score_support = 0.0
        else:  # NEUTRAL
            relevance_score_support = 1 - relevance_result["score"]

        # =====================================================
        # 2️⃣ Relevance (Support ↔ Opposing)
        # =====================================================
        relevance_result2 = nli_relevance_check(
            self.nli_pipeline,
            argument,
            opposing_argument
        )

        if relevance_result2["label"] == "ENTAILMENT":
            relevance_score_contra = 1 - relevance_result2["score"]
        elif relevance_result2["label"] == "NEUTRAL":
            relevance_score_contra = 0.0
        else:  # CONTRADICTION
            relevance_score_contra = relevance_result2["score"]

        # =====================================================
        # 3️⃣ Semantic Similarity
        # =====================================================
        similarity_score = similarity_check(
            self.similarity_model,
            topic,
            argument
        )

        return {
            "avg_logprob": avg_logprob,
            "relevance_score_support": relevance_score_support,
            "relevance_score_contra": relevance_score_contra,
            "similarity_score": similarity_score
        }


In [51]:
from typing import List
def prepare_data(data) -> List[:]:
    feature_extractor = FeatureExtractor()
    training_data = []
    for d in data:
        topic= d["topic"]
        truth_score=d["WA"]*d["stance_WA"]
        features=feature_extractor.extract_features(topic)
        training_data.append([topic,features,truth_score ])
    return training_data
        

In [ ]:
training_data=prepare_data(ds["train"])


🚀 FeatureExtractor using device: cuda


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


📥 Loading Model Weights & Tokenizer...
📥 Loading tokenizer: mistralai/Mistral-7B-Instruct-v0.1


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]